In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 模拟输入：batch=2, 序列长度=6, embedding维度=3
torch.manual_seed(123)
batch = torch.randn(2, 6, 3)

class SimpleMultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, num_heads):
        super().__init__()
        assert d_out % num_heads == 0, "d_out必须能被num_heads整除"
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        # Q/K/V 映射
        self.W_q = nn.Linear(d_in, d_out)
        self.W_k = nn.Linear(d_in, d_out)
        self.W_v = nn.Linear(d_in, d_out)

        # 输出拼接后的投影
        self.out_proj = nn.Linear(d_out, d_out)

    def forward(self, x):
        b, seq_len, _ = x.shape
        Q = self.W_q(x).view(b, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.W_k(x).view(b, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.W_v(x).view(b, seq_len, self.num_heads, self.head_dim).transpose(1, 2)

        # 注意力得分 (b, num_heads, seq_len, seq_len)
        attn_scores = Q @ K.transpose(-2, -1) / (self.head_dim ** 0.5)
        attn_weights = F.softmax(attn_scores, dim=-1)

        print("Head 1 注意力权重：\n", attn_weights[0,0].detach())
        if self.num_heads > 1:
            print("Head 2 注意力权重：\n", attn_weights[0,1].detach())

        # 上下文向量
        context = attn_weights @ V
        context = context.transpose(1, 2).contiguous().view(b, seq_len, -1)
        return self.out_proj(context)

# 测试
mha = SimpleMultiHeadAttention(d_in=3, d_out=4, num_heads=2)
out = mha(batch)
print("输出维度：", out.shape)


Head 1 注意力权重：
 tensor([[0.1651, 0.1663, 0.1637, 0.1634, 0.1771, 0.1644],
        [0.2141, 0.1167, 0.1445, 0.1662, 0.1067, 0.2517],
        [0.2264, 0.1032, 0.1358, 0.1627, 0.0930, 0.2789],
        [0.1584, 0.1733, 0.1661, 0.1626, 0.1851, 0.1544],
        [0.1260, 0.2079, 0.1737, 0.1548, 0.2274, 0.1102],
        [0.1399, 0.1905, 0.1679, 0.1563, 0.2168, 0.1285]])
Head 2 注意力权重：
 tensor([[0.1530, 0.1864, 0.1798, 0.1714, 0.1636, 0.1458],
        [0.1945, 0.1175, 0.1247, 0.1757, 0.1554, 0.2322],
        [0.1785, 0.1516, 0.1567, 0.1589, 0.1698, 0.1845],
        [0.1771, 0.1311, 0.1310, 0.2056, 0.1461, 0.2092],
        [0.1952, 0.0902, 0.0945, 0.2152, 0.1290, 0.2758],
        [0.1440, 0.1979, 0.1858, 0.1787, 0.1590, 0.1346]])
输出维度： torch.Size([2, 6, 4])


1. d_out 的作用

控制 注意力层输出的 embedding 维度。

就像一个「瓶颈」：你想让输出向量是几维，就设定 d_out。

在 Transformer 里，通常 d_out = d_model，也就是整个模型的隐层维度（比如 512, 768, 1024…）。

d_out 决定了 每个 token 最终的表示有多“宽”。

 2. num_heads 的作用

控制 并行的注意力子空间数量。

每个 head = 一套独立的 Q/K/V 投影，它会捕捉 不同类型的关系。

类比：

单头注意力：只有一个视角去看关系。

多头注意力：多个视角并行，有的 head 学语法，有的 head 学语义，有的 head 学长距离依赖。